## Import libraries

In [ ]:
!pip install transformers[sentencepiece] datasets rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 17.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 109.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.0/379.0 KB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.5/94.5 KB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.7/139.7 KB 19.8 MB/s eta 0

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

import time
import pandas as pd
import numpy as np

from transformers import pipeline,set_seed
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer

from datasets import load_metric

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import nltk
from nltk.tokenize import sent_tokenize

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Read dataset

In [ ]:
train = pd.read_csv('/content/gdrive/MyDrive/Text Mining Exam/X-Sum/Data/train_sport.csv', sep=';')
test = pd.read_csv('/content/gdrive/MyDrive/Text Mining Exam/X-Sum/Data/test_sport.csv', sep=';')
valid = pd.read_csv('/content/gdrive/MyDrive/Text Mining Exam/X-Sum/Data/validation_sport.csv', sep=';')

In [ ]:
train.head()

,document,summary,id,URL,category
0,Ferrari appeared in a position to challenge un...,Lewis Hamilton stormed to pole position at the...,35951548,http://web.archive.org/web/20160403040918/http...,formula1
1,Simone Favaro got the crucial try with the las...,Defending Pro12 champions Glasgow Warriors bag...,34540833,http://web.archive.org/web/20151017072732/http...,rugby-union
2,Belgian cyclist Demoitie died after a collisio...,Welsh cyclist Luke Rowe says changes to the sp...,35932467,http://web.archive.org/web/20160404024929/http...,wales
3,"Gundogan, 26, told BBC Sport he ""can see the f...",Manchester City midfielder Ilkay Gundogan says...,40758845,http://web.archive.org/web/20170729030028/http...,football
4,"The Bath-born player, 28, has made 36 appearan...",Newport Gwent Dragons number eight Ed Jackson ...,38900884,http://web.archive.org/web/20170208052041/http...,rugby-union



#Definition alternative Pipelines

In [ ]:
sample_text = train["document"][1]
print(sample_text)

summaries = {}

Simone Favaro got the crucial try with the last move of the game, following earlier touchdowns by Chris Fusaro, Zander Fagerson and Junior Bulumakau.
Rynard Landman and Ashton Hewitt got a try in either half for the Dragons.
Glasgow showed far superior strength in depth as they took control of a messy match in the second period.
Home coach Gregor Townsend gave a debut to powerhouse Fijian-born Wallaby wing Taqele Naiyaravoro, and centre Alex Dunbar returned from long-term injury, while the Dragons gave first starts of the season to wing Aled Brew and hooker Elliot Dee.
Glasgow lost hooker Pat McArthur to an early shoulder injury but took advantage of their first pressure when Rory Clegg slotted over a penalty on 12 minutes.
It took 24 minutes for a disjointed game to produce a try as Sarel Pretorius sniped from close range and Landman forced his way over for Jason Tovey to convert - although it was the lock's last contribution as he departed with a chest injury shortly afterwards.
Glas

##Baseline

In [ ]:
def baseline_summary_three_sent(text):
    return " ".join(sent_tokenize(text)[:3])

summaries['baseline'] = baseline_summary_three_sent(sample_text) 
summaries['baseline']

'Simone Favaro got the crucial try with the last move of the game, following earlier touchdowns by Chris Fusaro, Zander Fagerson and Junior Bulumakau. Rynard Landman and Ashton Hewitt got a try in either half for the Dragons. Glasgow showed far superior strength in depth as they took control of a messy match in the second period.'

##T5


In [ ]:
pipe_t5 = pipeline('summarization', model='t5-small')
pipe_t5_out = pipe_t5(sample_text))

Token indices sequence length is longer than the specified maximum sequence length for this model (908 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
pipe_t5_out

[{'summary_text': 'Gregor Townsend gave a debut to powerhouse fijian-born wallaby wing Taqele Naiyaravoro . the dragons gave first starts of the season to wing aled Brew and hooker Elliot Dee . it took 24 minutes for a disjointed game to produce a try .'}]


In [ ]:
summaries['t5'] = " ".join(sent_tokenize(pipe_t5_out[0]['summary_text']))
summaries['t5']

Gregor Townsend gave a debut to powerhouse fijian-born wallaby wing Taqele Naiyaravoro . the dragons gave first starts of the season to wing aled Brew and hooker Elliot Dee . it took 24 minutes for a disjointed game to produce a try .


##BART


In [ ]:
pipe_bart = pipeline("summarization", model="facebook/bart-large-cnn")
pipe_bart_out = pipe_bart(sample_text)

In [ ]:
print(pipe_bart(sample_text, max_length=130, min_length=30, do_sample=False))

[{'summary_text': 'Rynard Landman and Ashton Hewitt score tries in either half for Dragons. Chris Fusaro, Zander Fagerson and Junior Bulumakau score tries for Warriors. Simone Favaro scores crucial try with the last move of the game.'}]


In [ ]:
summaries["bart"] = " ".join(sent_tokenize(pipe_bart_out[0]["summary_text"]))
summaries["bart"]

'Rynard Landman and Ashton Hewitt score tries in either half for the Dragons. Chris Fusaro, Zander Fagerson and Junior Bulumakau score tries for the Warriors. Simone Favaro scores crucial try with the last move of the game. Dragons director of rugby Lyn Jones says errors cost his side.'

##PEGASUS

In [ ]:
pipe_pegasus = pipeline('summarization', model="google/pegasus-xsum")
pipe_pegasus_out = pipe_pegasus(sample_text[1:1000])

In [ ]:
pipe_pegasus_out

[{'summary_text': 'Glasgow Warriors began their Pro12 season with a bonus-point win over Newport Gwent Dragons at Scotstoun.'}]

In [ ]:
summaries["pegasus"] = pipe_pegasus_out[0]["summary_text"].replace("<n>", " ")
summaries["pegasus"] 

'Glasgow Warriors began their Pro12 season with a bonus-point win over Newport Gwent Dragons at Scotstoun.'

# Comparing Different Summaries

In [ ]:
print("GROUND TRUTH")

print(train["summary"][1])

for model_name in summaries:
    print(model_name.upper())
    print(summaries[model_name])

GROUND TRUTH
Defending Pro12 champions Glasgow Warriors bagged a late bonus-point victory over the Dragons despite a host of absentees and two yellow cards.
BASELINE
Simone Favaro got the crucial try with the last move of the game, following earlier touchdowns by Chris Fusaro, Zander Fagerson and Junior Bulumakau. Rynard Landman and Ashton Hewitt got a try in either half for the Dragons. Glasgow showed far superior strength in depth as they took control of a messy match in the second period.
T5
Gregor Townsend gave a debut to powerhouse fijian-born wallaby wing Taqele Naiyaravoro . the dragons gave first starts of the season to wing aled Brew and hooker Elliot Dee . it took 24 minutes for a disjointed game to produce a try .
BART
Rynard Landman and Ashton Hewitt score tries in either half for the Dragons. Chris Fusaro, Zander Fagerson and Junior Bulumakau score tries for the Warriors. Simone Favaro scores crucial try with the last move of the game. Dragons director of rugby Lyn Jones s

##ROUGE


In [ ]:
rouge_metric = load_metric('rouge')

In [ ]:
import timeit
start = timeit.timeit()

rouge_names= ["rouge1","rouge2","rougeL","rougeLsum"]
reference = train["summary"][1]
records = []

for model_name in summaries:
    print(model_name)
    rouge_metric.add(prediction=summaries[model_name], reference=reference)
    score = rouge_metric.compute()
    rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
    print('rouge_dict ', rouge_dict)
    records.append(rouge_dict)

pd.DataFrame.from_records(records, index=summaries.keys())

end = timeit.timeit()
print(end - start)

baseline
rouge_dict  {'rouge1': 0.175, 'rouge2': 0.025641025641025637, 'rougeL': 0.09999999999999999, 'rougeLsum': 0.09999999999999999}
t5
rouge_dict  {'rouge1': 0.1875, 'rouge2': 0.03225806451612903, 'rougeL': 0.15625, 'rougeLsum': 0.15625}
bart
rouge_dict  {'rouge1': 0.14084507042253522, 'rouge2': 0.02898550724637681, 'rougeL': 0.11267605633802817, 'rougeLsum': 0.11267605633802817}
pegasus
rouge_dict  {'rouge1': 0.39999999999999997, 'rouge2': 0.10526315789473685, 'rougeL': 0.35, 'rougeLsum': 0.35}
-0.0002864789998966444


#Evaluation on the test set 

## Baseline

In [ ]:
def calculate_metric_on_baseline_test_ds(dataset, metric, column_text='document', column_summary='summary'):

    summaries = [baseline_summary_three_sent(text) for text in dataset[column_text]]
    metric.add_batch(predictions=summaries, references=dataset[column_summary])
    return metric.compute()

In [ ]:
score = calculate_metric_on_baseline_test_ds(test, rouge_metric)
rouge_dict_baseline = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame.from_dict(rouge_dict_baseline, orient='index', columns=['baseline']).T

,rouge1,rouge2,rougeL,rougeLsum
baseline,0.168064,0.020025,0.106801,0.107651


In [ ]:
# baseline	0.168064	0.020025	0.106801	0.107651

## T5

In [ ]:
def calculate_metric_on_t5_test_ds(dataset, metric, column_text='document', column_summary='summary'):

    summaries = [" ".join(sent_tokenize(pipe_t5(text)[0]["summary_text"])) for text in dataset[column_text]]
    metric.add_batch(predictions=summaries, references=dataset[column_summary])
    return metric.compute()

In [ ]:
start = time.time()

score = calculate_metric_on_t5_test_ds(test, rouge_metric)
rouge_dict_t5 = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame.from_dict(rouge_dict_t5, orient='index', columns=['t5']).T

end = time.time()
print(end - start)

In [ ]:
pd.DataFrame.from_dict(rouge_dict_t5, orient='index', columns=['t5']).T

In [ ]:
	# t5	0.170973	0.02345	0.117049	0.117024

## BART

In [ ]:
def calculate_metric_on_bart_test_ds(dataset, metric, column_text='document', column_summary='summary'):

    summaries = ["\n".join(sent_tokenize(pipe_bart(text[:1000])[0]["summary_text"])) for text in dataset[column_text]]
    metric.add_batch(predictions=summaries, references=dataset[column_summary])
    return metric.compute()

In [ ]:
start = time.time()

score = calculate_metric_on_bart_test_ds(test, rouge_metric)
rouge_dict_bart = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame.from_dict(rouge_dict_bart, orient='index', columns=['Bart']).T

end = time.time()
print(end - start)

In [ ]:
pd.DataFrame.from_dict(rouge_dict_bart, orient='index', columns=['Bart']).T

,rouge1,rouge2,rougeL,rougeLsum
Bart,0.203238,0.041465,0.134517,0.165915


In [ ]:
# Bart	0.203238	0.041465	0.134517	0.165915

## Pegasus

In [ ]:
def calculate_metric_on_pegagus_test_ds(dataset, metric, column_text='document', column_summary='summary'):

    summaries = [pipe_pegasus(text[:1000])[0]["summary_text"].replace(" .", ".\n") for text in dataset[column_text]]
    metric.add_batch(predictions=summaries, references=dataset[column_summary])
    return metric.compute()

In [ ]:
start = time.time()

score = calculate_metric_on_pegagus_test_ds(test, rouge_metric)
rouge_dict_pegasus = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame.from_dict(rouge_dict_pegasus, orient='index', columns=['Pegasus']).T

end = time.time()
print(end - start)

In [ ]:
pd.DataFrame.from_dict(rouge_dict_pegasus, orient='index', columns=['Pegasus']).T

,rouge1,rouge2,rougeL,rougeLsum
Pegasus,0.471652,0.268646,0.41213,0.414099


In [ ]:
# Pegasus	0.471652	0.268646	0.41213	0.414099

Il modello Pegasus risulta il migliore tra quelli considerati. Si procede ad applicare il fine tuning del modello sul dataset in uso.